In [97]:
#System and standard inputs
import sys
import importlib
import timeit
import time
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [2]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [101]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
#OrderBook class import as written by Kang
import orderbook.orderbook
importlib.reload(orderbook.orderbook)
from orderbook.orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

In [121]:
#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#            each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kang/Data/TSLA_2015-01-02_34200000_57600000_orderbook_10.csv'

In [4]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_message_50.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_orderbook_50.csv", header=None)
orderbookLevels=50

In [5]:
data=jnp.array(df_ob.iloc[0,:]).reshape(int(orderbookLevels*2),2) ##Careful - converts to int32 in jnp array, meaning the filler values get truncated.
newarr=jnp.zeros((int(orderbookLevels*2),7))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:orderbookLevels*4:2,1].set(-1).at[1:orderbookLevels*4:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200.00000000)


df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]

ordersJNP=jnp.array(df_orders)
print(ordersJNP.shape)

(290737, 7)


In [104]:
horizon=3345

orderbook_1=JaxOrderBook(price_levels=100,orderQueueLen=100)
trades_init=orderbook_1.process_orders_array(initOB)
t1=time.time()
trades_msgs=orderbook_1.process_orders_array(ordersJNP[0:horizon-1]).block_until_ready()
t2=time.time()
#trades_msgs=orderbook_1.process_orders_array(ordersJNP[horizon-1:horizon])

In [105]:
t2-t1

0.5394079685211182

In [45]:
trades_msgs=orderbook_1.process_orders_array(ordersJNP[horizon-1:horizon])

In [46]:
trades_msgs

Array([[[-1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1.]]], dtype=float32)

In [53]:
MINPRICE=-1.410065407e9
myOB=orderbook_1.get_L2_state()
L2OB=jnp.array(df_ob.iloc[horizon-1]).astype(float)
L2OB=L2OB.at[L2OB==MINPRICE].set(-1)
#print(L2OB.reshape(50,4))
print(myOB.reshape(100,4))
#print((myOB.reshape(100,4)[0:50,:]-L2OB.reshape(50,4)).astype(int))#L2OB#-myOB

[[ 2.2169e+06  1.9900e+02  2.2137e+06  1.0000e+02]
 [ 2.2174e+06  1.0000e+02  2.2136e+06  1.0000e+02]
 [ 2.2176e+06  1.0000e+02  2.2134e+06  1.0000e+02]
 [ 2.2178e+06  5.0000e+02  2.2122e+06  2.0000e+02]
 [ 2.2198e+06  1.0000e+02  2.2120e+06  9.0000e+02]
 [ 2.2200e+06  7.0000e+00  2.2114e+06  1.0000e+02]
 [ 2.2224e+06  1.0000e+02  2.2111e+06  4.0000e+02]
 [ 2.2230e+06  1.0000e+02  2.2106e+06  2.5000e+01]
 [ 2.2240e+06  1.0000e+02  2.2104e+06  1.0000e+02]
 [ 2.2260e+06  2.0000e+02  2.2102e+06  1.8000e+01]
 [ 2.2272e+06  1.0000e+02  2.2101e+06  2.0000e+01]
 [ 2.2280e+06  1.0000e+02  2.2100e+06  1.2520e+03]
 [ 2.2288e+06  1.0000e+02  2.2088e+06  1.0000e+02]
 [ 2.2294e+06  1.0000e+02  2.2066e+06  1.0000e+02]
 [ 2.2304e+06  1.0000e+02  2.2064e+06  1.0000e+02]
 [ 2.2311e+06  1.0000e+02  2.2051e+06  1.0000e+02]
 [ 2.2320e+06  2.0000e+02  2.2050e+06  1.1150e+03]
 [ 2.2335e+06  2.3000e+01  2.2048e+06  1.0000e+02]
 [ 2.2343e+06  1.0000e+02  2.2040e+06  2.2000e+02]
 [ 2.2356e+06  1.0000e+02  2.20

In [83]:
print(ordersJNP[horizon-6][5])
print(df_msg.iloc[horizon-6,:])
ordersJNP[horizon-8:horizon+2]

19238772.0
0    35024.455145
1               1
2        19238773
3             100
4         2213400
5               1
6             NaN
Name: 3339, dtype: object


Array([[ 1.0000000e+00, -1.0000000e+00,  1.0000000e+02,  2.2174000e+06,
         0.0000000e+00,  1.9238772e+07,  3.5024457e+04],
       [ 3.0000000e+00,  1.0000000e+00,  1.0000000e+02,  2.2117000e+06,
         0.0000000e+00,  1.9104924e+07,  3.5024457e+04],
       [ 1.0000000e+00,  1.0000000e+00,  1.0000000e+02,  2.2134000e+06,
         0.0000000e+00,  1.9238772e+07,  3.5024457e+04],
       [ 3.0000000e+00,  1.0000000e+00,  1.0000000e+02,  2.2135000e+06,
         0.0000000e+00,  1.9238704e+07,  3.5024457e+04],
       [ 1.0000000e+00,  1.0000000e+00,  1.0000000e+02,  2.2136000e+06,
         0.0000000e+00,  1.9238784e+07,  3.5024457e+04],
       [ 3.0000000e+00,  1.0000000e+00,  1.0000000e+02,  2.2136000e+06,
         0.0000000e+00,  1.9238784e+07,  3.5024461e+04],
       [ 1.0000000e+00,  1.0000000e+00,  1.0000000e+02,  2.2137000e+06,
         0.0000000e+00,  1.9238828e+07,  3.5024461e+04],
       [ 3.0000000e+00,  1.0000000e+00,  1.0000000e+02,  2.2136000e+06,
         0.0000000e+00,  

In [90]:
a=jnp.array(19238773.0,dtype='float64')
a#.astype('float32')

/var/folders/sp/5klxb3311218qqklkfj7y6nc0000gp/T/ipykernel_18712/2046636918.py:1: UserWarning: Explicitly requested dtype float64 requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  a=jnp.array(19238773.0,dtype='float64')


Array(19238772., dtype=float32)

In [54]:
orderbook_1.orderbook_array[1,1,:,:]

Array([[ 1.0000000e+02,  2.2136000e+06,  0.0000000e+00,  1.9238772e+07,
         3.5024457e+04],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.000

In [460]:
trades=jnp.ones([5,6])*-1.
job.processOrder(orderbook_1.orderbook_array,ordersJNP[horizon-1])[0][1,1,:,:]

Array([[ 1.0000000e+02,  2.2134000e+06,  0.0000000e+00,  1.9238772e+07,
         3.5024457e+04],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00],
       [-1.0000000e+00, -1.000

## Load the original orderbook implementation on the CPU

In [107]:
orderbook_2=cpuOrderBook()

In [108]:
limit_orders = []
order_id_list = [orderbook.orderbook.INITID + i for i in range(100)]
for i in range(50):
    trade_id = 10086
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'ask', 
          'quantity' : df_ob.iloc[0,4* i + 1], 
          'price' : df_ob.iloc[0,4*i],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
for i in range(50):
    trade_id = 10087
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'bid', 
          'quantity' : df_ob.iloc[0,4* i + 3], 
          'price' : df_ob.iloc[0,4*i+2],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
# Add init orders to order book
for order in limit_orders:
    # breakpoint()
    trades, order_id = orderbook_2.process_order(order, True, False)  

In [109]:
##Taken largely from the Example.py file in the orderbook repository for the cpu

t1=time.time()
for index in range(horizon):
    l1 = df_msg.iloc[index,:]
    ttype = l1[1] 
    side = 'bid' if l1[5] ==1 else 'ask'
    quantity = l1[3]
    price = l1[4]
    trade_id = l1[2] # not sure, in the data it is order id
    order_id = trade_id
    timestamp = str(l1[0])
    message = {'type': ttype,'side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
    orderbook_2.processOrder(message,True,False)

t2=time.time()
        
print(message)

got to the ask arm
got to the ask arm
got to the no ID issue
Quantity    40  |  Price  2257500  |  Order_ID   15000009  |  Time 34200.000000001

Trying to cancel partial order
{'type': 'delete', 'side': 'ask', 'quantity': 20, 'price': 2257500, 'trade_id': 5097802, 'timestamp': '34200.501657651', 'order_id': 15000009}
got to the ask arm
got to the no ID issue
Quantity   300  |  Price  2237200  |  Order_ID   15000002  |  Time 34200.000000001

Deleting whole order
got to the bid arm
got to the no ID issue
Quantity   300  |  Price  2226400  |  Order_ID   15000000  |  Time 34200.000000001

Deleting whole order
got to the bid arm
got to the no ID issue
Quantity   200  |  Price  2222200  |  Order_ID   15000001  |  Time 34200.000000001
Quantity     1  |  Price  2222200  |  Order_ID    2584610  |  Time 34200.078873974

Deleting whole order
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got t

In [110]:
t2-t1

0.22741413116455078

In [410]:
from itertools import zip_longest
cpuOB=jnp.array(list(zip_longest(*orderbook_2.get_L2_state(), fillvalue=-1)))

In [386]:
print(cpuOB[:50,:])
print(myOB.reshape(100,4))
print((cpuOB[:50,:]-myOB.reshape(100,4)[0:50,:].astype('int32')))

[[2215000       2 2213200     100]
 [2216900     299 2212200     200]
 [2217600     100 2212000     900]
 [2217800     500 2211700     100]
 [2219800     100 2211400     100]
 [2220000       7 2211100     400]
 [2222400     100 2210600      25]
 [2223000     100 2210400     100]
 [2224000     200 2210200      18]
 [2225600     100 2210100      20]
 [2226000     200 2210000    1252]
 [2227200     100 2208800     100]
 [2228000     100 2206600     100]
 [2228800     100 2206400     100]
 [2230400     100 2205100     100]
 [2231100     100 2205000    1115]
 [2232000     200 2204800     100]
 [2233500      23 2204000     220]
 [2234300     100 2203600     100]
 [2235600     100 2203400      50]
 [2236800     100 2202500       1]
 [2237800     100 2201600      25]
 [2239200     100 2201500     200]
 [2240000     400 2201100     100]
 [2240400      25 2201000       8]
 [2241000     100 2200300     125]
 [2241500     250 2200100     150]
 [2242500      20 2200000    4288]
 [2244600     100 21